<a target="_blank" href="https://colab.research.google.com/github/wakusoftware/curso-ml-espanol/blob/master/C1%20-%20Aprendizaje%20Supervisado/W2/C1_W2_Lab04_IngCars_RegPol.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Setup para Colab
Si estás corriendo este Notebook en Google Colab corre la celda de abajo, de lo contrario ignórala.

In [ ]:
!git clone https://github.com/wakusoftware/curso-ml-espanol.git

%cd curso-ml-espanol/C1 - Aprendizaje Supervisado/W2/

!cp lab_utils_common.py /content/

!cp lab_utils_multi.py /content/

!cp deeplearning.mplstyle /content/

!cp -r data /content/

!cp -r images /content/

%cd /content/

!rm -rf curso-ml-espanol/

# Laboratorio: Ingeniería de características y regresión polinómica

## Objetivos
En este laboratorio realizarás:
- exploración de la ingeniería de características y la regresión polinómica, lo que te permite usar la maquinaria de la regresión lineal para ajustar funciones muy complicadas, incluso muy no lineales.

## Herramientas
Utilizarás la función desarrollada en laboratorios anteriores, así como matplotlib y NumPy.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lab_utils_multi import zscore_normalize_features, run_gradient_descent_feng
np.set_printoptions(precision=2)  # precisión reducida en la visualización de arreglos de numpy

<a name='FeatureEng'></a>
# Visión General de la Ingeniería de Características y la Regresión Polinómica

De manera predeterminada, la regresión lineal ofrece un medio para construir modelos de la forma:
$$f_{\mathbf{w},b} = w_0x_0 + w_1x_1+ ... + w_{n-1}x_{n-1} + b \tag{1}$$
¿Qué sucede si tus características/datos no son lineales o son combinaciones de características? Por ejemplo, los precios de las viviendas no tienden a ser lineales con respecto al área habitable, sino que penalizan las casas muy pequeñas o muy grandes, resultando en las curvas mostradas en el gráfico anterior. ¿Cómo podemos usar la maquinaria de la regresión lineal para ajustar esta curva? Recuerda, la 'maquinaria' que tenemos es la capacidad de modificar los parámetros $\mathbf{w}$, $\mathbf{b}$ en (1) para 'ajustar' la ecuación a los datos de entrenamiento. Sin embargo, ningún ajuste de $\mathbf{w}$, $\mathbf{b}$ en (1) logrará un ajuste a una curva no lineal.

<a name='PolynomialFeatures'></a>
## Características Polinómicas

Anteriormente consideramos un escenario donde los datos eran no lineales. Vamos a intentar usar lo que sabemos hasta ahora para ajustar una curva no lineal. Empezaremos con una cuadrática simple: $y = 1+x^2$

Estás familiarizado con todas las rutinas que estamos utilizando. Están disponibles en el archivo lab_utils.py para revisión. Usaremos [`np.c_[..]`](https://numpy.org/doc/stable/reference/generated/numpy.c_.html) que es una rutina de NumPy para concatenar a lo largo del límite de la columna.

In [ ]:
# crear datos objetivo
x = np.arange(0, 20, 1)
y = 1 + x**2
X = x.reshape(-1, 1)

model_w, model_b = run_gradient_descent_feng(X, y, iterations=1000, alpha = 1e-2)

plt.scatter(x, y, marker='x', c='r', label="Valor real"); plt.title("sin ingeniería de características")
plt.plot(x, X@model_w + model_b, label="Valor predicho");  plt.xlabel("X"); plt.ylabel("y"); plt.legend(); plt.show()

Bueno, como se esperaba, no es un buen ajuste. Lo que se necesita es algo como $y= w_0x_0^2 + b$, o una **característica polinómica**.
Para lograr esto, puedes modificar los *datos de entrada* para *ingeniar* las características necesarias. Si cambias los datos originales por una versión que eleve al cuadrado el valor de $x$, entonces puedes lograr $y= w_0x_0^2 + b$. Vamos a intentarlo. Cambia `X` por `X**2` a continuación:

In [ ]:
# crear datos objetivo
x = np.arange(0, 20, 1)
y = 1 + x**2

# Ingeniar características
X = x**2      # <-- característica modificada agregada

In [ ]:
X = X.reshape(-1, 1)  # X debe ser una matriz 2-D
model_w, model_b = run_gradient_descent_feng(X, y, iterations=10000, alpha = 1e-5)

plt.scatter(x, y, marker='x', c='r', label="Valor actual"); plt.title("Característica agregada x**2")
plt.plot(x, np.dot(X,model_w) + model_b, label="Valor predicho"); plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()

¡Genial! Casi un ajuste perfecto. Observa los valores de $\mathbf{w}$ y b impresos justo arriba del gráfico: `w, b encontrados por descenso de gradiente: w: [1.], b: 0.0490`. El descenso de gradiente modificó nuestros valores iniciales de $\mathbf{w}, b$ para ser (1.0, 0.049) o un modelo de $y=1*x_0^2+0.049$, muy cercano a nuestro objetivo de $y=1*x_0^2+1$. Si lo ejecutaras más tiempo, podría ser una mejor coincidencia.

### Seleccionando Características
<a name='GDF'></a>
Anteriormente, sabíamos que se requería un término $x^2$. No siempre puede ser obvio qué características son necesarias. Se podrían añadir una variedad de características potenciales para intentar encontrar las más útiles. Por ejemplo, ¿qué pasaría si en lugar de eso hubiéramos intentado: $y=w_0x_0 + w_1x_1^2 + w_2x_2^3+b$?

Ejecuta las siguientes celdas.

In [ ]:
# crear datos objetivo
x = np.arange(0, 20, 1)
y = x**2

# ingeniar características
X = np.c_[x, x**2, x**3]   # <-- característica modificada agregada

In [ ]:
model_w, model_b = run_gradient_descent_feng(X, y, iteraciones=10000, alpha=1e-7)

plt.scatter(x, y, marcador='x', c='r', etiqueta="Valor actual"); plt.title("Características x, x**2, x**3")
plt.plot(x, X@model_w + model_b, etiqueta="Valor predicho"); plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()

Observa el valor de $\mathbf{w}$, `[0.08 0.54 0.03]` y b es `0.0106`. Esto implica que el modelo después de ajustar/entrenar es:
$$ 0.08x + 0.54x^2 + 0.03x^3 + 0.0106 $$
El descenso de gradiente ha enfatizado los datos que mejor se ajustan a los datos de $x^2$ aumentando el término $w_1$ en relación con los demás. Si se ejecutara durante mucho tiempo, continuaría reduciendo el impacto de los otros términos.
> El descenso de gradiente está seleccionando las 'características correctas' para nosotros al enfatizar su parámetro asociado.

Revisemos esta idea:
- Inicialmente, las características fueron reescaladas para que sean comparables entre sí.
- Un valor de peso menor implica una característica menos importante/correcta, y en extremo, cuando el peso se vuelve cero o muy cercano a cero, la característica asociada es útil para ajustar el modelo a los datos.
- Arriba, después del ajuste, el peso asociado con la característica de $x^2$ es mucho mayor que los pesos para $x$ o $x^3$, ya que es la más útil para ajustar los datos.

### Una Visión Alternativa
Arriba, las características polinómicas se eligieron según cuán bien coincidían con los datos objetivo. Otra forma de pensar en esto es notar que todavía estamos usando regresión lineal una vez que hemos creado nuevas características. Dado esto, las mejores características serán lineales en relación con el objetivo. Esto se entiende mejor con un ejemplo.

In [ ]:
# crear datos objetivo
x = np.arange(0, 20, 1)
y = x**2

# ingeniar características
X = np.c_[x, x**2, x**3]   # <-- característica ingenierizada agregada
X_features = ['x', 'x^2', 'x^3']

In [ ]:
fig,ax=plt.subplots(1, 3, figsize=(12, 3), sharey=True)
for i in range(len(ax)):
    ax[i].scatter(X[:,i],y)
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("y")
plt.show()

Arriba, es claro que la característica $x^2$ mapeada contra el valor objetivo $y$ es lineal. Entonces, la regresión lineal puede fácilmente generar un modelo usando esa característica.


### Escalado de características
Como se describió en el último laboratorio, si el conjunto de datos tiene características con escalas significativamente diferentes, se debe aplicar el escalado de características para acelerar el descenso de gradiente. En el ejemplo anterior, hay $x$, $x^2$ y $x^3$ que naturalmente tendrán escalas muy diferentes. Apliquemos la normalización Z-score a nuestro ejemplo.

In [ ]:
# crear datos objetivo
x = np.arange(0,20,1)
X = np.c_[x, x**2, x**3]
print(f"Rango de pico a pico por columna en X crudo: {np.ptp(X, axis=0)}")

# añadir normalización media
X = zscore_normalize_features(X)
print(f"Rango de pico a pico por columna en X normalizado: {np.ptp(X, axis=0)}")

Ahora podemos intentarlo nuevamente con un valor más agresivo de alpha:

In [ ]:
x = np.arange(0,20,1)
y = x**2

X = np.c_[x, x**2, x**3]
X = zscore_normalize_features(X)

model_w, model_b = run_gradient_descent_feng(X, y, iteraciones=100000, alpha=1e-1)

plt.scatter(x, y, marcador='x', c='r', etiqueta="Valor actual"); plt.title("Característica normalizada x, x**2, x**3")
plt.plot(x, X@model_w + model_b, etiqueta="Valor predicho"); plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()

El escalado de características permite que esto converja mucho más rápido.
Nota nuevamente los valores de $\mathbf{w}$. El término $w_1$, que es el término $x^2$, es el más enfatizado. El descenso de gradiente ha eliminado casi por completo el término $x^3$.

### Funciones Complejas
Con la ingeniería de características, incluso funciones bastante complejas pueden ser modeladas:

In [ ]:
x = np.arange(0,20,1)
y = np.cos(x/2)

X = np.c_[x, x**2, x**3, x**4, x**5, x**6, x**7, x**8, x**9, x**10, x**11, x**12, x**13]
X = zscore_normalize_features(X)

model_w, model_b = run_gradient_descent_feng(X, y, iteraciones=1000000, alpha = 1e-1)

plt.scatter(x, y, marcador='x', c='r', etiqueta="Valor actual"); plt.title("Característica normalizada x, x**2, x**3")
plt.plot(x, X@model_w + model_b, etiqueta="Valor predicho"); plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()

## ¡Felicidades!
En este laboratorio:
- aprendiste cómo la regresión lineal puede modelar funciones complejas, incluso altamente no lineales, mediante la ingeniería de características
- reconociste que es importante aplicar el escalado de características al realizar la ingeniería de características